## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1660
+------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+------------------+-------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|            coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|             res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+------------------+-------------------+-----------+-----------+------+--------------------+------+
|1040.1244553796428|-191.20163471725976|-140.21296811651058|     70.1|BSBSSSBB| 32.5575| -82.9036|       TOBS|0.41991797987969487|0.39986009272794104|0.3886385581013813|0.1944650

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [19]:
#extract longitude and latitude for each station
feature='coeff_2'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


station    year     coeff_2
0  USC00092839  2004.0 -191.201635
1  USC00084731  1950.0  359.575952
2  USC00099186  1979.0  -81.408203
3  USC00092966  1968.0 -318.683740
4  USC00084394  1963.0 -112.367711

In [20]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station  USC00083470  USC00084393  USC00084394  USC00084731  USC00085099  \
year                                                                       
1901.0           NaN          NaN          NaN   310.984522          NaN   
1902.0           NaN          NaN          NaN   191.588824          NaN   
1903.0           NaN   555.800433          NaN   264.018651          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN   -91.969161          NaN          NaN          NaN   
1906.0           NaN    95.126936          NaN          NaN          NaN   
1907.0           NaN   393.636175          NaN   641.236087          NaN   
1908.0           NaN   -55.189371          NaN   489.651799          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN   411.914975   -63.986004   

station  USC00085275  USC00089640  USC00090010  USC00090090  USC00090131  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0    363.289006          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00098351  USC00098476  USC00098496  USC00098972  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1906.0      ...               NaN          NaN          NaN          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   
1908.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1910.0      ...               NaN          NaN          NaN          NaN   

station  USC00098974  USC00099020  USC00099186  USC00099451  USW00013861  \
year                                                                       
1901.0           NaN          NaN   -17.958058          NaN          NaN   
1902.0           NaN  -307.991448          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN  -514.258938          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station  USW00093845  
year                  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  
1907.0           NaN  
1908.0           NaN  
1909.0           NaN  
1910.0           NaN  

[10 rows x 55 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [21]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

total_rms = RMS(year_station_table)
rms_minus_station = RMS(tbl_minus_station)
station_effect = (total_rms - rms_minus_station) / total_rms * 100
rms_minus_year = RMS(tbl_minus_year)
year_effect = (total_rms- rms_minus_year) / total_rms * 100
print 'total RMS                   = ',total_rms
print 'RMS removing mean-by-station= ', rms_minus_station, '%f%%' % station_effect
print 'RMS removing mean-by-year   = ',rms_minus_year, '%f%%' % year_effect

total RMS                   =  212.114913682
RMS removing mean-by-station=  191.675900704 9.635821%
RMS removing mean-by-year   =  140.912370859 33.567910%


In [22]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)

initial RMS= 212.114913682
0 after removing mean by stations= 191.675900704
0 after removing mean by year    = 119.919898371
1 after removing mean by stations= 117.810282092
1 after removing mean by year    = 117.560350869
2 after removing mean by stations= 117.496139486
2 after removing mean by year    = 117.469613381
3 after removing mean by stations= 117.456341673
3 after removing mean by year    = 117.449151283
4 after removing mean by stations= 117.445102599
4 after removing mean by year    = 117.442776612
